In [ ]:
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import requests
import os

In [ ]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName", "shapeID") to create properties for output file
#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-CHN-Ningbo-ADM1.geojson'
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM2/boundary-CRI-San_Jose-ADM2.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
print(Districts.limit(1).getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]
ADMlevel = basename.split('-')[-1]

Areaofinterest =  AOIname # "ETH-Addis_Ababa" ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
unitofanalysis =  ADMlevel # "ADM4" ## options: "ADM0" (country), "ADM1" (state), "Metro" (metropolitan region), "ADM2" (municipality), "ADM3" (subcity/locality), "ADM4"(ward/neighborhood), ideally align with https://www.geoboundaries.org/index.html#getdata

print(Areaofinterest)
print(unitofanalysis)

In [ ]:
## create map
Map = geemap.Map(height="400px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)
Map.addLayer(Districts,{},"Districts")

In [ ]:
## Calculate and add to map vegetation and water cover rasters

NDVIthreshold = 0.4 # decimal
NDWIthreshold = 0.3 # decimal


s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  return image.addBands(ndwi)

gw2021 = s2.filterDate('2021-01-01', '2021-12-31').map(addNDVI).map(addNDWI)
gw2020 = s2.filterDate('2020-01-01', '2020-12-31').map(addNDVI).map(addNDWI)
gw2019 = s2.filterDate('2019-01-01', '2019-12-31').map(addNDVI).map(addNDWI)
gw2018 = s2.filterDate('2018-01-01', '2018-12-31').map(addNDVI).map(addNDWI)
gw2017 = s2.filterDate('2017-01-01', '2017-12-31').map(addNDVI).map(addNDWI)
gw2016 = s2.filterDate('2016-01-01', '2016-12-31').map(addNDVI).map(addNDWI)

g2021 = gw2021.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2021).rename('time_start')).float();
g2020 = gw2020.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2020).rename('time_start')).float();
g2019 = gw2019.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2019).rename('time_start')).float();
g2018 = gw2018.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2018).rename('time_start')).float();
g2017 = gw2017.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2017).rename('time_start')).float();
g2016 = gw2016.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(2016).rename('time_start')).float();

g2021mask = g2021.updateMask(g2021.select('NDVI').gte(NDVIthreshold))
g2020mask = g2020.updateMask(g2020.select('NDVI').gte(NDVIthreshold))
g2019mask = g2019.updateMask(g2019.select('NDVI').gte(NDVIthreshold))
g2018mask = g2018.updateMask(g2019.select('NDVI').gte(NDVIthreshold))
g2017mask = g2017.updateMask(g2017.select('NDVI').gte(NDVIthreshold))
g2016mask = g2016.updateMask(g2016.select('NDVI').gte(NDVIthreshold))

w2021 = gw2021.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2021).rename('time_start')).float()
w2020 = gw2020.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2020).rename('time_start')).float()
w2019 = gw2019.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2019).rename('time_start')).float()
w2018 = gw2018.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2018).rename('time_start')).float()
w2017 = gw2017.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2017).rename('time_start')).float()
w2016 = gw2016.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(2016).rename('time_start')).float()

w2021mask = w2021.updateMask(w2021.select('NDWI').gte(NDWIthreshold))
w2020mask = w2020.updateMask(w2020.select('NDWI').gte(NDWIthreshold))
w2019mask = w2019.updateMask(w2019.select('NDWI').gte(NDWIthreshold))
w2018mask = w2018.updateMask(w2019.select('NDWI').gte(NDWIthreshold))
w2017mask = w2017.updateMask(w2017.select('NDWI').gte(NDWIthreshold))
w2016mask = w2016.updateMask(w2016.select('NDWI').gte(NDWIthreshold))

## Mask out water pixels from vegetation layers
g2021mask = g2021mask.updateMask(w2021mask.unmask().Not())
g2020mask = g2020mask.updateMask(w2020mask.unmask().Not())
g2019mask = g2019mask.updateMask(w2019mask.unmask().Not())
g2018mask = g2018mask.updateMask(w2018mask.unmask().Not())
g2017mask = g2017mask.updateMask(w2017mask.unmask().Not())
g2016mask = g2016mask.updateMask(w2016mask.unmask().Not())

g2021 = g2021.updateMask(w2021mask.unmask().Not())
g2020 = g2020.updateMask(w2020mask.unmask().Not())
g2019 = g2019.updateMask(w2019mask.unmask().Not())
g2018 = g2018.updateMask(w2018mask.unmask().Not())
g2017 = g2017.updateMask(w2017mask.unmask().Not())
g2016 = g2016.updateMask(w2016mask.unmask().Not())


ndviParams = {"bands": ['NDVI'], min: -1, max: 1, "palette": ['blue', 'white', 'green']};
#Map.addLayer(g2016mask.select('NDVI'), ndviParams, 'Vegetation 2016: NDVI composite masked at >'+str(NDVIthreshold)+' (Sentinel)');
#Map.addLayer(g2017mask.select('NDVI'), ndviParams, 'Vegetation 2017: NDVI composite masked at >'+str(NDVIthreshold)+' (Sentinel)');
Map.addLayer(g2021mask.select('NDVI'), ndviParams, 'Vegetation 2021: NDVI composite masked at >'+str(NDVIthreshold)+' (Sentinel)');

ndwiParams = {"bands": ['NDWI'], min: -1, max: 1, "palette": ['yellow', 'blue', 'blue']};
#Map.addLayer(w2016mask.select('NDWI'), ndwiParams, 'Water 2016: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');
#Map.addLayer(w2017mask.select('NDWI'), ndwiParams, 'Water 2017: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');
#Map.addLayer(w2018mask.select('NDWI'), ndwiParams, 'Water 2018: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');
#Map.addLayer(w2019mask.select('NDWI'), ndwiParams, 'Water 2019: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');
#Map.addLayer(w2020mask.select('NDWI'), ndwiParams, 'Water 2020: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');
Map.addLayer(w2021mask.select('NDWI'), ndwiParams, 'Water 2021: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');


## create a time band function for the trend analysis.
#def createtimeband(image): 
#    return image.addBands(image.metadata('system:time_start').long())

#gtrend = ee.ImageCollection.fromImages([g2016mask,g2017mask,g2018mask,g2019mask,g2020mask,g2021mask])
gtrend = ee.ImageCollection.fromImages([g2016,g2017,g2018,g2019,g2020,g2021])
print(gtrend.getInfo())
#wtrend = ee.ImageCollection.fromImages([w2016mask,w2017mask,w2018mask,w2019mask,w2020mask,w2021mask])
wtrend = ee.ImageCollection.fromImages([w2016,w2017,w2018,w2019,w2020,w2021])



In [ ]:
# Create linear fit trend line for 6 years of data
glinearfit = ee.Image(gtrend.select(['time_start','NDVI']).reduce(ee.Reducer.linearFit()));
print(glinearfit.getInfo())

glfLimit = glinearfit.select('scale').multiply(ee.Image(6)).updateMask(glinearfit.select('scale').gte(0.03).Or(glinearfit.select('scale').lte(-0.03)))#.updateMask(glinearfit.select('offset').gte(-0.1));
glfLimitLoss = glfLimit.updateMask(glfLimit.lt(0))
glfLimitGain = glfLimit.updateMask(glfLimit.gt(0))

#Map.addLayer(glfLimit,{"min": -1, "max": 1, "palette":["red","yellow","green"]},"Veg Trend");

# Create linear fit trend line for 6 years of data
wlinearfit = ee.Image(wtrend.select(['time_start','NDWI']).reduce(ee.Reducer.linearFit()));
print(wlinearfit.getInfo())

wlfLimit = wlinearfit.select('scale').multiply(ee.Image(6)).updateMask(wlinearfit.select('scale').gte(0.03).Or(wlinearfit.select('scale').lte(-0.03)));#.updateMask(wlinearfit.select('offset').gte(-0.1));

wateranyyearMask = ee.Image(0).where(w2016mask.select('NDWI').unmask().add(w2017mask.select('NDWI').unmask()).add(w2018mask.select('NDWI').unmask()).add(w2019mask.select('NDWI').unmask()).add(w2020mask.select('NDWI').unmask()).add(w2021mask.select('NDWI').unmask()).neq(0),1)
#Map.addLayer(wateranyyearMask,{},'water any year mask')
wlfLimitanyyearwater = wlfLimit.updateMask(wateranyyearMask)
wlfLimitanyyearwaterLoss = wlfLimitanyyearwater.updateMask(wlfLimitanyyearwater.lt(0))
wlfLimitanyyearwaterGain = wlfLimitanyyearwater.updateMask(wlfLimitanyyearwater.gt(0))
                                                          
#Map.addLayer(wlfLimitanyyearwater,{"min": -1, "max": 1, "palette":['brown','white','blue']},"Water Trend");
#Map.addLayer(wlfLimitanyyearwaterLoss,{"min": -1, "max": 1, "palette":['brown','white','blue']},"Water Trend Loss");
#Map.addLayer(wlfLimitanyyearwaterGain,{"min": -1, "max": 1, "palette":['brown','white','blue']},"Water Trend Gain");



In [ ]:
# Apply significance test
# https://developers.google.com/earth-engine/tutorials/community/nonparametric-trends
# https://code.earthengine.google.com/bce3dc00c56df4246c5d32f5fcccf5c7
#//////////////////////////////////////singificance test///////////////////////////////////////////////////////////////

def senum(lfx):
  def senumwrap(Im):
      esty=lfx.select('scale').multiply(Im.select('time_start')).add(lfx.select('offset'))
      diff=Im.select('NDVI').subtract(esty)
      pow=diff.multiply(diff)
      return (pow)
  return (senumwrap)

def sedenom(mosaicmeanx):
  def sedenomwrap(Im):
    diff=Im.select('time_start').subtract(mosaicmeanx.select('time_start'));
    pow=diff.multiply(diff);
    return (pow)
  return (sedenomwrap)

#/////green normalized Difference///////
mosaicmean=gtrend.mean();
mosaicNum=gtrend.map(senum(glinearfit)).sum();
mosaicDenom=gtrend.map(sedenom(mosaicmean)).sum();
StdDev=mosaicNum.divide(mosaicDenom);
#Standard Error - from 6 samples
se= StdDev.divide(6).sqrt();
#Test Statistic
gT=glinearfit.select('scale').divide(se);

def senum(lfx):
  def senumwrap(Im):
      esty=lfx.select('scale').multiply(Im.select('time_start')).add(lfx.select('offset'))
      diff=Im.select('NDWI').subtract(esty)
      pow=diff.multiply(diff)
      return (pow)
  return (senumwrap)

#/////water normalized Difference///////
mosaicmean=wtrend.mean();
mosaicNum=wtrend.map(senum(wlinearfit)).sum();
mosaicDenom=wtrend.map(sedenom(mosaicmean)).sum();
StdDev=mosaicNum.divide(mosaicDenom);
#Standard Error - from 6 samples
se= StdDev.divide(6).sqrt();
#Test Statistic
wT=wlinearfit.select('scale').divide(se);

# https://en.wikipedia.org/wiki/Error_function#Cumulative_distribution_function
def eeCdf(t):
  return ee.Image(0.5).multiply(ee.Image(1).add(ee.Image(t).divide(ee.Image(2).sqrt()).erf()));

def invCdf(p):
  return ee.Image(2).sqrt().multiply(ee.Image(p).multiply(2).subtract(1).erfInv());

# Compute P-values.
gP = ee.Image(1).subtract(eeCdf(gT.abs()));
wP = ee.Image(1).subtract(eeCdf(wT.abs()));


# Pixels that can have the null hypothesis (there is no trend) rejected.
# Specifically, if the true trend is zero, there would be less than 5%
# chance of randomly obtaining the observed result (that there is a trend).
glf=glfLimit.updateMask(gP.lte(0.025));
Map.addLayer(glf, {'bands':['scale'],'min': -1, 'max': 1,'palette':['red','yellow','green']}, 'significant veg trends');

glfLoss = glf.updateMask(glf.lt(0))
glfGain = glf.updateMask(glf.gt(0))

wlf=wlfLimit.updateMask(wP.lte(0.025));
wlfanyyearwater = wlf.updateMask(wateranyyearMask)
Map.addLayer(wlfanyyearwater, {'bands':['scale'],'min': -1, 'max': 1,'palette':['brown','white','blue']}, 'significant water trends');

wlfanyyearwaterLoss = wlfanyyearwater.updateMask(wlfanyyearwater.lt(0))
wlfanyyearwaterGain = wlfanyyearwater.updateMask(wlfanyyearwater.gt(0))
#Map.addLayer(wlfanyyearwaterLoss, {'bands':['scale'],'min': -1, 'max': 1,'palette':['brown','white','blue']}, 'significant water loss');
#Map.addLayer(wlfanyyearwaterGain, {'bands':['scale'],'min': -1, 'max': 1,'palette':['brown','white','blue']}, 'significant water gain');


Map
#//----------------------------------------------------------------------------------------//


In [ ]:
stackedForCount = ( 
    w2021.select('NDWI').rename('pixels')
  .addBands(g2016mask.select('NDVI').rename('NDVI2016'))
  .addBands(g2021mask.select('NDVI').rename('NDVI2021'))
  .addBands(w2016mask.select('NDWI').rename('NDWI2016'))
  .addBands(w2021mask.select('NDWI').rename('NDWI2021'))
  .addBands(glfLoss.rename('greenLoss'))
  .addBands(glfGain.rename('greenGain'))
  .addBands(wlfanyyearwaterLoss.rename('waterLoss'))
  .addBands(wlfanyyearwaterGain.rename('waterGain'))
 )

print(stackedForCount.getInfo())

In [ ]:
histo=stackedForCount.reduceRegions(
  reducer= ee.Reducer.count(),
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)
#print('histo:', histo.limit(1).getInfo())

In [ ]:
def count_to_percent(feat):
    feat=ee.Feature(feat)
    
    FeatArea = feat.area(0.001).multiply(0.000001)
    cityID = Areaofinterest
    geo_level = unitofanalysis
    #geo_level = feat.getString("shapeID").split('-').getString(1)
    geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    #geo_name = feat.getString("shapeName").split(' ').join('_')
    geo_id = ee.String(cityID+"-").cat(geo_name)
    year = 2020
    source = "Sentinel-2, WRI"
    
    # convert 10m pixel count to KM2 
    convert10mtoKM2 = ee.Number(100).multiply(ee.Number(0.000001))
    FeatAreafromPixelsKM2 = feat.getNumber('pixels').multiply(convert10mtoKM2)
    
    vegArea2021KM2 = feat.getNumber('NDVI2021').multiply(convert10mtoKM2)
    waterArea2021KM2 = feat.getNumber('NDWI2021').multiply(convert10mtoKM2)
    vegPct2021 = ee.Number(vegArea2021KM2).divide(FeatAreafromPixelsKM2)
    waterPct2021 = ee.Number(waterArea2021KM2).divide(FeatAreafromPixelsKM2)
    vegGainKM2 = feat.getNumber('greenGain').multiply(convert10mtoKM2)
    vegLossKM2 = feat.getNumber('greenLoss').multiply(convert10mtoKM2)
    waterGainKM2 = feat.getNumber('waterGain').multiply(convert10mtoKM2)
    waterLossKM2 = feat.getNumber('waterLoss').multiply(convert10mtoKM2)
    
    return feat.set({
        'TotalareaKM2': FeatArea,
        'TotalareaFromPixelsKM2': FeatArea,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'year':year,
        'source':source,
        'vegArea2021KM2': vegArea2021KM2,
        'waterArea2021KM2':waterArea2021KM2,
        'vegPct2021':vegPct2021,
        'waterPct2021':waterPct2021,
        'vegGainKM2':vegGainKM2,
        'vegLossKM2':vegLossKM2,
        'waterGainKM2':waterGainKM2,
        'waterLossKM2':waterLossKM2
    })

In [ ]:
## update FeatureCollection with percents

VegWat_pcts=histo.map(count_to_percent)

#print('Veg and water cover by Districts',VegWat_pcts.limit(1).getInfo());

In [ ]:
## render on map percent veg cover for each feature from feature collection

Vpctfills = ee.Image().byte().paint(**{'featureCollection': VegWat_pcts,'color': 'vegPct2021'})

fillspalette = ['red', 'green']
Map.addLayer(Vpctfills, {'palette': fillspalette,'min':0,'max':1}, '% vegetation cover 2021', True, 0.65)
Map

In [ ]:
## select properties to keep, sort features and create data frame to display properties
VegWat_pcts = VegWat_pcts.select([
    'TotalareaKM2',
    'TotalareaFromPixelsKM2',
    'geo_level',
    'geo_name',
    'geo_id',
    'year',
    'source',
    'vegArea2021KM2',
    'waterArea2021KM2',
    'vegPct2021',
    'waterPct2021',
    'vegGainKM2',
    'vegLossKM2',
    'waterGainKM2',
    'waterLossKM2'
]).sort('vegPct2021', False) #
#print('Veg and water cover by Districts',VegWat_pcts.limit(1).getInfo());

In [ ]:
df = geemap.ee_to_geopandas(VegWat_pcts)
df = df.sort_values(by=['vegPct2021'],axis='index',ascending=False)
df